In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/eelco/dev/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=a6d297d7880c44f4a77a5e943d97d4b8 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=d9b7f12c251845bab562b6fd2a6a5b2b in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# create datasets

In [4]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.16it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [5]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.34it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/eade7aa03ee743f1a02c10a9efd4747e.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=c4dc910bdbe6401792e97e733e79c528 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_490: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_490
ca1d2e544e944eea8b2d18439d683d3a
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

SyftInfo: Node Landed!


In [7]:
client_low.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [8]:
from syft.client.syncing import get_sync_state, compare_states

low_state = get_sync_state(client_low)
high_state = get_sync_state(client_high)

diff_state = compare_states(low_state, high_state)

In [9]:
diff_state.diffs

[syft.client.syncing.Diff, syft.client.syncing.Diff]

In [10]:
# state.objs_to_sync
from syft.client.syncing import resolve
low_items_to_sync, high_items_to_sync = resolve(diff_state)

```python
class Request:
  id: str = 3fd1bf045e1046f2966a70e899de2446
  request_time: str = 2024-02-16 15:53:41
  updated_at: str = None
  status: str = RequestStatus.PENDING
  changes: str = ['Request to change <b>compute_mean</b> (Pool Id: <b>default-pool</b>) to permission <b>RequestStatus.APPROVED</b>. No nested requests']
  requesting_user_verify_key: str = 1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


 approve


```python
class UserCode
    id: UID = a036b42d54a34031a72706a45300d20e
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


 approve


- What do we do with properties here?
  - just set them to None

In [11]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [12]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [13]:
def create_actionobject(action_object, client):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

In [14]:
def sync_items(items, node, client):
    for item in items:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client)

        else:
            stash = get_stash_for_item(item, node)

            set_obj_ids(item, client.id, client.verify_key)

            res = stash.delete_by_uid(node.python_node.verify_key, item.id)

            res = stash.set(node.python_node.verify_key, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)

In [15]:
low_items_to_sync

[]

In [16]:
sync_items(low_items_to_sync, node_low, client_low)

sync_items(high_items_to_sync, node_high, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


In [17]:
low_state = get_sync_state(client_low)
high_state = get_sync_state(client_high)

diff_state = compare_states(low_state, high_state)

diff_state.diffs

[syft.client.syncing.Diff, syft.client.syncing.Diff]

# Run code high and sync back result

In [18]:
client_high._fetch_api(client_high.credentials)

In [19]:
data_high = client_high.datasets[0].assets[0]

In [20]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 794b8c884c464b3aacf2e95af2a26eaf
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

a036b42d54a34031a72706a45300d20e
override True


In [21]:
# wait for the result
job_high.wait()

```python
Pointer
```
17.0

In [22]:
job_high.result.get()

17.0

In [23]:
low_state = get_sync_state(client_low)
high_state = get_sync_state(client_high)

diff_state_2 = compare_states(low_state, high_state)

diff_state_2.diffs

[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

In [24]:
# items_to_sync = state_2.objs_to_sync()
# items_to_sync.append(job_high.result)
low_items_to_sync, high_items_to_sync = resolve(diff_state_2)

```python
class Job:
    id: UID = 794b8c884c464b3aacf2e95af2a26eaf
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


```python
class SyftLog:
  id: str = 3415c2fd3d474c9784a323a680fa2ef4

```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


```python
Pointer
```
17.0

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


 approve


In [25]:
sync_items(low_items_to_sync, node_low, client_low)

created syft.service.job.job_stash.Job
created syft.service.log.log.SyftLog
Created SyftError: Arg: Could not find item with uid 06bb86aff91b490da5194b57cf6e3dfb, "06bb86aff91b490da5194b57cf6e3dfb not in <class 'syft.store.sqlite_document_store.SQLiteBackingStore'>" must be Union not str


In [26]:
# low_node_state = client_low.get_sync_state() -> just a table
# high_node_state = client_high.get_sync_state()
# (low_new_state, high_new_state) = resolve_state(low_node_state, high_node_state)

In [27]:
low_state = get_sync_state(client_low)
high_state = get_sync_state(client_high)

diff_state_3 = compare_states(low_state, high_state)

diff_state_3.diffs

[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

# Run code low

In [28]:
private_high.mean()

17.0

In [29]:
job_low = client_low.code.compute_mean(data=data_low, blocking=False)

a036b42d54a34031a72706a45300d20e


SyftWarning: There are existing jobs for this user code, returning the latest one

In [30]:
job_low

```python
class Job:
    id: UID = 794b8c884c464b3aacf2e95af2a26eaf
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

In [31]:
job_low.result

```python
Pointer
```
17.0

# TODO

- Displaying tables (diff table + state table)
    - Improving table flexibility 
- We're not syncing correct IDs with syncing ActionObjects
- We're not syncing outputpolicy
- Incorrect datetimes between updated_at on requests
- When we create multiple changes in requests, we have to sync those as well
- Object hierarchy is not as clean as we want (both Jobs and Requests are at the top of the hierarchy)
- TODO Merge with Koen's permissions